In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
from reason.core import AbstractTerm, Variable
from reason.vampire.translator import to_fof
from reason.parser import Parser
parser = Parser()

In [5]:
print(parser.ogc.create_lark_code())



logic_simple: logic_level_7

logic_level_0: abstract_term | "(" logic_simple ")" -> brk

OP2_1.1: "∩"
OP2_2.1: "∪"
OP2_3.1: "∈"| "="
OP1_4.1: "~"
OP2_5.1: "and"
OP2_6.1: "∨"| "or"
OP2_7.1: "→"| "⇒"| "⟷"| "⇔"

logic_level_1: logic_level_1 OP2_1 logic_level_0 -> _op2 | logic_level_0
logic_level_2: logic_level_2 OP2_2 logic_level_1 -> _op2 | logic_level_1
logic_level_3: logic_level_3 OP2_3 logic_level_2 -> _op2 | logic_level_2
logic_level_4: OP1_4 logic_level_3 -> _op1 | logic_level_3
logic_level_5: logic_level_5 OP2_5 logic_level_4 -> _op2 | logic_level_4
logic_level_6: logic_level_6 OP2_6 logic_level_5 -> _op2 | logic_level_5
logic_level_7: logic_level_7 OP2_7 logic_level_6 -> _op2 | logic_level_6




In [14]:
parser("P(x) or Q(x)")

OR(P(x), Q(x))

In [7]:
from reason.vampire import Vampire
vampire_prover = Vampire(parser, verbose=False)

In [23]:
from reason.core.theory import Theory
ZFC = Theory(parser, vampire_prover)

ZFC.add_const("∅")

ZFC.add_axiom("∀(x, y) x = y ⟷ (∀(z) z ∈ x ⟷ z ∈ y)", name="a0")
ZFC.add_axiom("∀(x) ~(x ∈ ∅)", name="a1")
ZFC.add_axiom("empty(e) ⟷ (∀(x) ~(x ∈ e))", name="d1")
ZFC.add_axiom("∀(x, z) z ∈ s(x) ⟷ z = x", name="a3")
ZFC.add_axiom("∀(x, y, z) z ∈ x ∪ y ⟷ z ∈ x ∨ z ∈ y", name='a4')
ZFC.add_axiom("∀(x, y, z) z ∈ x ∩ y ⟷ z ∈ x ∧ z ∈ y", name='a5')

In [51]:
ZFC("∀(x, y) empty(x) ∧ empty(y) → x = y")

True

In [52]:
ZFC("empty(x) → x = ∅")

True

In [53]:
ZFC("s(x) = s(y) → x = y")

True

In [56]:
ZFC("∀(x, y, z) z = s(x) ∪ s(y) → x ∈ z ∧ y ∈ z")

True

In [55]:
ZFC("~(s(x) ∩ s(y) = ∅) → x = y")

KeyboardInterrupt: 

In [28]:
ZFC("~(s(x) ∩ s(y) = ∅) → (∃(z) z ∈ s(x) ∧ z ∈ s(y))")

True

In [47]:
ZFC("∀(x, y, z) z ∈ s(x) ∧ z ∈ s(y) → z = x ∧ z = y")

True

In [ ]:
ZFC("∀(x, y, z) x = z ∧ z = y → x = y")

In [43]:
ZFC("(x = z ∧ z = y) → (((x = z ∧ z = y) ∧ ((∀(k) k ∈ x ⟷ k ∈ z) ∧ (∀(k) k ∈ z ⟷ k ∈ y) → (∀(k) (k ∈ x ⟷ k ∈ y)))) → x = y)")

True

In [ ]:
ZFC("∀(x) ~(x = s(x)))